# Cortes
>$n$: Quantidade de demandas  
>$m$: Quantidade de padrões de corte  

## Variáveis
>$x_j$: Quantidade de bobinas, para $j = 1,\ldots, m$  

## Parâmetros
>$M$: Compromento da bobina matriz.  
>$D_i$: Demanda das bobinas menores, para $i = 1,\ldots, n$  
>$P_{ij}$: Padrões de corte, para $i = 1,\ldots, n$ e $j = 1,\ldots, m$  

## Modelo  

\begin{align*}
    \hbox{min} \ \ 
        & \sum_{j=1}^m x_i \\
        
    \hbox{s.a.} \ \ 
        & \sum_{j=1}^m P_{ij} x_j \geq D_i && i = 1,\ldots, n \\
        & P_{ij} \geq 0                    && i = 1,\ldots, n \,;\, j = 1,\ldots, m \\
        & D_i \geq 0                       && i = 1,\ldots, n \\
        & M   \geq 0
\end{align*}

In [39]:
from itertools import product


def cut_patterns(M: int, C: list[int]) -> list[list[int]]:
    corte_minimo = M - min(C)
    cortes = product(*(range(M//i + 1) for i in C))
    x = [i for i in cortes if corte_minimo < sum(a*b for a, b in zip(i, C)) <= M]

    return x

In [56]:
import pandas as pd
from IPython.display import display

def create_table(M: int, C: list[int]):
    patterns = cut_patterns(M, C)
    df = pd.DataFrame(patterns, columns=C)
    df["Resto"] = M - df.mul(C).sum(axis=1)
    display(df)
    
    return df.to_string()

In [46]:
from pymprog import *

def modelo_cortes(M, C, D):
    P = [*zip(*cut_patterns(M, C))]

    n = len(D)
    m = len(P[0])

    begin("cortes")

    x = var("x", m, int)

    minimize( sum(x) )

    for i in range(n):
        sum(P[i][j] * x[j] for j in range(m)) >= D[i]

    solve()

    obj = vobj()

    end()

    return obj, x, P

In [ ]:
import os

for file in os.listdir("./input/"):
    with open(f"input/{file}", "r") as f:
        M = int(f.readline())

        # descarta dado não utilizado
        f.readline()

        C, D = [], []
        for i in f.readlines():
            c, d = map(int, i.split())
            C.append(c)
            D.append(d)

        # create_table(M, C)
        obj, varx, P = modelo_cortes(M, C, D)
        P = [*zip(*P)]

        with open(f"output/{file}", "w") as f:
            f.write(create_table(M, C) + "\n\n")

            f.write(f"Tamanho da bobina matriz: {M}\n")
            f.write(f"Tamanho das bobinas menores: {C}\n")
            f.write(f"Demandas: {D}\n\n")

            f.write(f"Quantidade de bobinas usadas: {int(obj)}\n\n")

            desperdicio_total = 0

            for i, x in enumerate(varx):
                if x.primal:
                    resto = M - sum(a*b for a, b in zip(P[i], C))
                    desperdicio_total += int(x.primal) * resto

                    f.write(f"Padrão {i:>2}: {P[i]} resto {resto}\n")
                    f.write(f"Quantidade usada: {int(x.primal)}\n")
                    f.write(f"Quantidade desperdiçada: {int(x.primal) * resto}\n")
                    f.write(f"Quantidade satisfeita de cada demanda: {tuple(j * int(x.primal) for j in P[i])}\n\n")

            f.write(f"Quantidade total de papel gasto: {int(obj) * M}\n")
            f.write(f"Quantidade desperdiçada: {desperdicio_total}\n")